# Автоматическая обработка текстов 
## Домашнее задание 1 (Зятчин 396 и Зацепин 396)

В этом домашнем задании вам потребуется написать генератор описания прогноза погоды на следующую неделю в каком-нибудь городе. Домашнее задание состоит из трех частей:
1. Скачивание данных о состоянии погоды в городе 
2. Генерация описания прогноза
3. Творческая часть

Все три части можно считать независимыми – вы можете сделать одну или две из них, однако мы настоятельно советуем выполнить все три. Все инструкции по выполнению домашнего задания – ниже. 



### 1. Сбор данных

Для сбора информации о прогнозах погоды на 10 дней использовалось API сайта wunderground.com.

Были извлечены следующие данные:

* минимальная температура (в °C)
* максимальная температура (в °C)
* скорость ветра (в м/с)
* уровень осадков (в мм)

Собственно ниже представлен код отправки запроса и извлечения данных из формата json.

P.S. Многие известные сервисы дают бесплатные прогнозы только на 5-7 дней, поэтому для нашего задания пришлось поискать noname-сервис :)

In [2]:
import json
import numpy as np
import pandas as pd
import requests
import datetime
import pymorphy2
import calendar

In [3]:
CITY, CITY_ENG = 'Москва', 'Moscow'

def get_city():
    return CITY


MIN_TEMPERATURE = 'минимальная температура'
MAX_TEMPERATURE = 'максимальная температура'
WIND_SPEED = 'скорость ветра'
PRECIPITATION_LEVEL = 'уровень осадков'

In [4]:
forecast_url = 'http://api.wunderground.com/api/f80f6ff287475a26/forecast10day/q/RU/' + CITY_ENG + '.json'

In [5]:
forecast_data = requests.get(forecast_url).json()
daily10_forecast_data = forecast_data['forecast']['simpleforecast']['forecastday']

In [6]:
forecast_row_index = [MIN_TEMPERATURE, MAX_TEMPERATURE, WIND_SPEED, PRECIPITATION_LEVEL]
forecast_table = pd.DataFrame(np.zeros(shape=(4, 10), dtype=np.int32), index=forecast_row_index)

dates = []

for i in range(10):
    daily_forecast = daily10_forecast_data[i]

    forecast_table.set_value(forecast_row_index[0], i, daily_forecast['low']['celsius'])
    forecast_table.set_value(forecast_row_index[1], i, daily_forecast['high']['celsius'])
    forecast_table.set_value(forecast_row_index[2], i, daily_forecast['avewind']['kph'])
    
    qpf_daily = daily_forecast['qpf_day']['mm']
    if qpf_daily == None:
        qpf_daily = 0
    forecast_table.set_value(forecast_row_index[3], i, qpf_daily )
    
    day = datetime.date(daily_forecast['date']['year'], daily_forecast['date']['month'], daily_forecast['date']['day'])
    dates.append(day)
    
forecast_table.rename(columns=lambda x: dates[x], inplace=True)

In [7]:
forecast_table

,2018-03-15,2018-03-16,2018-03-17,2018-03-18,2018-03-19,2018-03-20,2018-03-21,2018-03-22,2018-03-23,2018-03-24
минимальная температура,-12,-19,-18,-13,-9,-11,-13,-13,-10,-9
максимальная температура,-2,-8,-9,-6,-2,-1,-5,-6,-4,-2
скорость ветра,21,18,14,13,14,13,21,21,18,14
уровень осадков,4,0,0,0,0,1,1,1,0,1


### 2. Генератор описания прогноза погоды


В следующих ячейках генерируются прогнозы по следующим компонентам:

* Похолодание / потепление
* Изменение скорости ветра
* Повышение / понижение уровня осадков

In [9]:
DAYS = forecast_table.columns

In [11]:
# index - одно из четырех значений, например MIN_TEMPERATURE
# day - число от 0 до 9
def get_value_for_day(index, day):
    return forecast_table.get_value(index, DAYS[day])

In [12]:
def get_weekday_in_russian(day):
    if day == 0:
        return 'понедельник'
    elif day == 1:
        return 'вторник'
    elif day == 2:
        return 'среда'
    elif day == 3:
        return 'четверг'
    elif day == 4:
        return 'пятница'
    elif day == 5:
        return 'суббота'
    elif day == 6:
        return 'воскресенье'
    assert(false)

In [13]:
def make_first_letter_uppercase(word):
    return word[0].upper() + word[1:]

In [14]:
morph = pymorphy2.MorphAnalyzer()

### Температурный прогноз:

In [16]:
def get_forecast_type_1(day1, day2):
    day1_word = morph.parse(get_weekday_in_russian(DAYS[day1].weekday()))[0].inflect({'ablt'}).word
    day2_word = morph.parse(get_weekday_in_russian(DAYS[day2].weekday()))[0].inflect({'accs'}).word

    city_word = make_first_letter_uppercase(morph.parse(get_city())[0].inflect({'datv'}).word)
    
    min_temp1 = get_value_for_day(MIN_TEMPERATURE, day1)
    max_temp1 = get_value_for_day(MAX_TEMPERATURE, day1)
    avg_temp1 = (max_temp1 + min_temp1) // 2.
    
    min_temp2 = get_value_for_day(MIN_TEMPERATURE, day2)
    max_temp2 = get_value_for_day(MAX_TEMPERATURE, day2)
    avg_temp2 = (max_temp2 + min_temp2) // 2.
    
    delta_temp = avg_temp2 - avg_temp1
        
    action = 'потеплеет'
    if delta_temp < 0:
        action = 'похолодает'
        delta_temp *= -1
        
    if delta_temp == 0:
        action = 'температура не изменится'
        TEMPLATE = "В {} в {} {} по сравнению c {}"
        return TEMPLATE.format(day2_word, city_word, action, day1_word)
    
    units = 'градус'
    correct_units = morph.parse(units)[0].make_agree_with_number(delta_temp).word
        
    TEMPLATE = "В {} в {} {} на {} {} по сравнению c {}"
    return TEMPLATE.format(day2_word,
                           city_word,
                           action,
                           int(delta_temp),
                           correct_units,
                           day1_word)

В воскресенье в Москве похолодает на 3 градуса по сравнению c четвергом


In [17]:
for i in range(0, 7):
    for j in range(i+1,7):
        print(get_forecast_type_1(i, j))

В пятницу в Москве похолодает на 7 градусов по сравнению c четвергом
В субботу в Москве похолодает на 7 градусов по сравнению c четвергом
В воскресенье в Москве похолодает на 3 градуса по сравнению c четвергом
В понедельник в Москве потеплеет на 1 градус по сравнению c четвергом
В вторник в Москве потеплеет на 1 градус по сравнению c четвергом
В среду в Москве похолодает на 2 градуса по сравнению c четвергом
В субботу в Москве температура не изменится по сравнению c пятницей
В воскресенье в Москве потеплеет на 4 градуса по сравнению c пятницей
В понедельник в Москве потеплеет на 8 градусов по сравнению c пятницей
В вторник в Москве потеплеет на 8 градусов по сравнению c пятницей
В среду в Москве потеплеет на 5 градусов по сравнению c пятницей
В воскресенье в Москве потеплеет на 4 градуса по сравнению c субботой
В понедельник в Москве потеплеет на 8 градусов по сравнению c субботой
В вторник в Москве потеплеет на 8 градусов по сравнению c субботой
В среду в Москве потеплеет на 5 градусо

### Прогнозирование скорости ветра:

In [25]:
def get_forecast_type_2(day1, day2):
    wind_speed_word = make_first_letter_uppercase(WIND_SPEED)
    
    day1_word = morph.parse(get_weekday_in_russian(DAYS[day1].weekday()))[0].inflect({'ablt'}).word
    day2_word = morph.parse(get_weekday_in_russian(DAYS[day2].weekday()))[0].inflect({'accs'}).word

    city_word = make_first_letter_uppercase(morph.parse(get_city())[0].inflect({'datv'}).word)
    
    wind_speed1 = get_value_for_day(WIND_SPEED, day1)
    wind_speed2 = get_value_for_day(WIND_SPEED, day2)
    
    delta_wind_speed = wind_speed2 - wind_speed1
    
    action = 'увеличится'
    if delta_wind_speed < 0.:
        delta_wind_speed *= -1.
        action = 'уменьшится'
        
    if delta_wind_speed == 0.:
        action = 'не изменится'
        TEMPLATE = "{} в {} в {} {} по сравнению c {}"
        return TEMPLATE.format(wind_speed_word, day2_word, city_word, action, day1_word)
    
    units = 'единица'
    correct_units = morph.parse(units)[0].make_agree_with_number(delta_wind_speed)
    
    # обработка единственного числа (delta_wind_speed = 1)
    if delta_wind_speed == 1.:
        correct_units = correct_units.inflect({'sing', 'accs'})
    
    correct_units_word = correct_units.word
    
    TEMPLATE = "{} в {} в {} {} на {} {} по сравнению c {}"
    return TEMPLATE.format(wind_speed_word, 
                           day2_word, 
                           city_word,
                           action,
                           int(delta_wind_speed),
                           correct_units_word,
                           day1_word)

In [19]:
for i in range(0, 7):
    for j in range(i+1,7):
        print(get_forecast_type_2(i, j))

Скорость ветра в пятницу в Москве уменьшится на 3 единицы по сравнению c четвергом
Скорость ветра в субботу в Москве уменьшится на 7 единиц по сравнению c четвергом
Скорость ветра в воскресенье в Москве уменьшится на 8 единиц по сравнению c четвергом
Скорость ветра в понедельник в Москве уменьшится на 7 единиц по сравнению c четвергом
Скорость ветра в вторник в Москве уменьшится на 8 единиц по сравнению c четвергом
Скорость ветра в среду в Москве не изменится по сравнению c четвергом
Скорость ветра в субботу в Москве уменьшится на 4 единицы по сравнению c пятницей
Скорость ветра в воскресенье в Москве уменьшится на 5 единиц по сравнению c пятницей
Скорость ветра в понедельник в Москве уменьшится на 4 единицы по сравнению c пятницей
Скорость ветра в вторник в Москве уменьшится на 5 единиц по сравнению c пятницей
Скорость ветра в среду в Москве увеличится на 3 единицы по сравнению c пятницей
Скорость ветра в воскресенье в Москве уменьшится на 1 единицу по сравнению c субботой
Скорость ве

### Прогнозирование уровня осадков

In [28]:
def get_forecast_type_3(day1, day2):
    precipitation_level_word = make_first_letter_uppercase(PRECIPITATION_LEVEL)
    
    day1_word = morph.parse(get_weekday_in_russian(DAYS[day1].weekday()))[0].inflect({'ablt'}).word
    day2_word = morph.parse(get_weekday_in_russian(DAYS[day2].weekday()))[0].inflect({'accs'}).word
    
    delta_days = day2 - day1 + 1
    time_unit = 'день'
    time_unit_word = morph.parse(time_unit)[0].make_agree_with_number(delta_days).word
    
    city_word = make_first_letter_uppercase(morph.parse(get_city())[0].inflect({'datv'}).word)
    
    precipitation_level1 = get_value_for_day(PRECIPITATION_LEVEL, day1)
    precipitation_level2 = get_value_for_day(PRECIPITATION_LEVEL, day2)
    
    delta_precipitation_level = precipitation_level2 - precipitation_level1
    
    action = 'повысится'
    if delta_precipitation_level < 0.:
        delta_precipitation_level *= -1.
        action = 'понизится'
        
    if delta_precipitation_level == 0.:
        action = 'не изменится'
        TEMPLATE = "{} в {} {} за {} {}"
        return TEMPLATE.format(precipitation_level_word, city_word, action, delta_days, time_unit_word )
    
    units = 'единица'
    correct_units = morph.parse(units)[0].make_agree_with_number(delta_precipitation_level)
    
    # обработка единственного числа (delta_precipitation_level = 1)
    if delta_precipitation_level == 1:
        correct_units = correct_units.inflect({'sing', 'accs'})
    
    correct_units_word = correct_units.word
    
    TEMPLATE = "{} в {} {} на {} {} за {} {}"
    return TEMPLATE.format(precipitation_level_word, 
                           city_word, 
                           action,
                           int(delta_precipitation_level),
                           correct_units_word,
                           delta_days,
                           time_unit_word)

In [23]:
for i in range(0, 7):
    for j in range(i+1,7):
        print(get_forecast_type_3(i, j))

Уровень осадков в Москве понизится на 4 единицы за 2 дня
Уровень осадков в Москве понизится на 4 единицы за 3 дня
Уровень осадков в Москве понизится на 4 единицы за 4 дня
Уровень осадков в Москве понизится на 4 единицы за 5 дней
Уровень осадков в Москве понизится на 3 единицы за 6 дней
Уровень осадков в Москве понизится на 3 единицы за 7 дней
Уровень осадков в Москве не изменится за 2 дня
Уровень осадков в Москве не изменится за 3 дня
Уровень осадков в Москве не изменится за 4 дня
Уровень осадков в Москве повысится на 1 единицу за 5 дней
Уровень осадков в Москве повысится на 1 единицу за 6 дней
Уровень осадков в Москве не изменится за 2 дня
Уровень осадков в Москве не изменится за 3 дня
Уровень осадков в Москве повысится на 1 единицу за 4 дня
Уровень осадков в Москве повысится на 1 единицу за 5 дней
Уровень осадков в Москве не изменится за 2 дня
Уровень осадков в Москве повысится на 1 единицу за 3 дня
Уровень осадков в Москве повысится на 1 единицу за 4 дня
Уровень осадков в Москве пов

### 3. Ответьте на вопросы

Вопрос 1. 

В каких других задачах (помимо описания прогноза погоды) может понадобиться генерировать текст по шаблонам? В каких задачах может понадобиться генерировать текст об изменении числовых показателей по шаблонам?

*  Достаточно распространенным примером является задача составления отчетности, особенно для формальных целей, например для получения справки или оформления документов.
*  В качестве дополнительных примеров конкретных областей применения, можно привести следующие области:
     * Финансовые отчеты. Например составление налоговой отчетности или предоставление информации для биржи. Также Можно рассмотреть финансовый рынок, для которого огромное количество людей составляют разнородные отчеты о всевозможных финансовых инструментах. Довольно много информации, засоряющей новостной фон, а ведь в целом, можно абсолютно просто автоматизировать составлвение отчетности и предоставлению клиенту отчета по шаблону. Например: "Рубль вырос по отношению к доллару и стоит 100500 долларов." Или "Акции "Газпрома" выросли на 500% и Ваша прибыль за сегодня составляет 100500 рублей." 
     * Спортивные отчеты. Например: "Спартак проиграл Ливерпулю 7 очков."


Вопрос 2. 

Шаблоны, которые вы использовали в этом задании, имеют фиксированную структуру. Фактически, ваша задача заключалась в том, чтобы подставить в шаблон число и согласовать единицы измерения с этим числом или подставить в шаблон название города и согласовать его с предлогом. Как можно разнообразить эти шаблоны? Как знание синтаксической структуры предложения может помочь в этой задаче? 

- Больше всего хотелось разнообразить отчеты обилием глаголов, потому что описание меняющихся величин можно делать не только с помощью слова "изменилось". Можно генерировать в шаблонах с некоторой вероятностью различные глаголы, или даже генерировать их в зависимости от величины, также можно вставлять оценочные характеристики. 

- Например можно комбинировать "незначительно" и "опустилось" или "невероятно" и "взлетело" и т.п. Кроме того, гибкость русского языка позволяет менять местами некоторые слова, что дает возможность генерировать чуть более разнообразные сообщения. Например "в Москве" можно было поставить практически куда угодно в шаблон (даже не меняя шаблон в коде). 

- Собственно знание синтаксической структуры позволяет определить правила генерации рандомных подстановок и перестановок в шаблоне. Можно сделать отдельный генератор шаблонов, с описанием того, что нужно подставлять. Например "Облигации _bond_name_ _adverb_option_ _action_name_ на _delta_price_ за _delta_time_", генератор вернет строку-шаблон "Облигации {} {} {} на {} за {}" и дескриптор [_bond_name_, _adverb_option_, _action_name_, _delta_price_, _delta_time_] и в коде нужно будет сгенерировать только описанные в дескрипторе сущности уже соответствующим генератором сущностей. Также добавить в дескриптор параметры согласования. 

- В качестве альтернативного подхода, можно создать большую базу шаблонов с описанием и случайно выбирать шаблон.